In [ ]:
import numpy as np
from astropy import units as u
import csv
import configparser
import os

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
from matplotlib.pyplot import figure, show, subplots
from matplotlib import pyplot as plt


In [ ]:
def lifesim_write_data(filepath,bandcenter,intensity):
    with open(filepath, 'w', newline='') as f:
        writer = csv.writer(f)
        for i in range(len(bandcenter)):
            string = [str(bandcenter[i]),str(intensity[i])]
            writer.writerow(string)

In [ ]:
# convert AGNI data to be used for LIFEsim
data_path = "../output/AGNI/AGNI_textfiles"
files = [f for f in os.listdir(data_path)]

# for i in range(len(files)):
#     image_path = data_path + "/" + files[i]
#     bandcenter = np.genfromtxt(image_path,skip_header=1,delimiter=",")[:,0]/1000 * u.micron
#     bandflux = np.genfromtxt(image_path,skip_header=1,delimiter=",")[:,1] * u.erg / (u.s * u.cm**2 * u.nm)
#     bandflux_converted = bandflux.to(u.Jy, equivalencies=u.spectral_density(bandcenter))
#     filename = '../output/AGNI/AGNI_spectra/AGNI_spectrum_' + str(round(coords[i][0],2)) + '_' + str(round(coords[i][1],2)) + '.'
#     fig.savefig(pngname)


In [ ]:
def agni_in_lifesim(agnifilename,distance,integrationtime):
    data_path = "../output/AGNI/AGNI_textfiles/" + agnifilename 
    bandcenter = np.genfromtxt(data_path,skip_header=1,delimiter=",")[:,0]/1000 * u.micron
    bandflux = np.genfromtxt(data_path,skip_header=1,delimiter=",")[:,1] * u.erg / (u.s * u.cm**2 * u.nm)
    bandflux_converted = bandflux.to(u.Jy, equivalencies=u.spectral_density(bandcenter))
    bandflux_distancecorrected = bandflux_converted*(6378/(distance*5.39E11))**2
    file_destination = "../output/LIFEsim/cleanspectra/" + "clean" + agnifilename[4:-4] + "_" + str(distance) + ".txt"
    with open(file_destination, 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(zip(bandcenter.value,bandflux_distancecorrected.value))
    config = configparser.ConfigParser(comment_prefixes="#",allow_no_value=True)
    config.read('GEMINI/init_simulation.cfg')
    config['simulation']['simulation_distance-to-planet'] = str(distance)
    config['simulation']['simulation_distance-to-star'] = str(distance)
    config['simulation']['simulation_integration-time'] = str(integrationtime)
    config['planet']['planet_pathtospectrum'] = "../" + file_destination
    with open('GEMINI/init_simulation.cfg', 'w') as configfile:
        config.write(configfile)
    
        



In [ ]:
sixteen_indices = [0,3,6,9,30,33,36,39,60,63,66,69,90,93,96,99]
for i in sixteen_indices:
    for j in [5,10,15,20]:
        agni_in_lifesim(files[i],j,36000)

In [ ]:
# save individual images
plt.ioff() # supress notebook output

data_path = "../output/LIFEsim/cleanspectra"
files = [f for f in os.listdir(data_path)]

for i in range(len(files)):
    image_path = data_path + "/" + files[i]
    bandcenter = np.genfromtxt(image_path)[:,0]
    bandflux = np.genfromtxt(image_path)[:,1]

    fig = figure(figsize=(8,6))
    frame1 = fig.add_subplot(1,1,1)
    frame1.plot(bandcenter,bandflux)
    frame1.set_xlabel('wavelength (microns)')
    frame1.set_ylabel('Janskys')
    #frame1.set_xscale('log')
    frame1.set_yscale('log')
    frame1.set_xlim(4,18.5)
    frame1.set_ylim(10**-6,10**-2)
    filename = '../output/LIFEsim/cleanspectra/images/' + files[i][:-4] + '.png'
    fig.savefig(filename)